In [1]:
import pickle

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalAveragePooling1D, Input, Activation, \
    BatchNormalization, MaxPooling1D, concatenate, LSTM

from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Recall
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf



def RedRNN(max_len=1600):
    now_pay_input_layer = Input((max_len, 257), name='now_pay')
    #embedding_layer = Embedding(257, embedding_size)(now_pay_input_layer)
    
    x = LSTM(32)(now_pay_input_layer)
    output_layer = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs=[now_pay_input_layer], outputs=output_layer)

In [2]:
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def preprocessing_payload(payloads, max_len=1600):
    tmp = []
    for payload in payloads:
        byte_payload = []
        for i in range(0, len(payload), 2):
            byte_payload.append(int(payload[i:i + 2], 16))
        tmp.append(byte_payload)
    tmp = pad_sequences(tmp, maxlen=max_len, padding='post', truncating='post', value=256)

    return to_categorical(tmp)

In [3]:
def extract(data):
    temp_payload = []
    temp_y = []
    for value in data.values():
        for j in range(len(value)):
            temp_payload.append(value[j][0])
            temp_y.append(2 - value[j][1])
    return temp_payload, temp_y

In [4]:
def myGenerator():
    data = pd.read_pickle("E:/full_data/from_2018_8_to_2019_1.pkl")
    X, y = extract(data)
    y = np.asarray(y, dtype=np.float32)
    length = len(X)
    bs = length//1000
    while 1:
        for i in range(1001): # 1443 * 1000 + remain[437] = 1,443,437 -> # of training samples
            tmp_X = preprocessing_payload(X[i*bs:(i+1)*bs])
            yield tmp_X, y[i*bs:(i+1)*bs]

In [5]:
def evaluateGenerator():
    data = pd.read_pickle("E:/full_data/from_2019_2_to_2019_3.pkl")
    X, y = extract(data)
    y = np.asarray(y, dtype=np.float32)
    length = len(X)
    bs = length//1000
    while 1:
        for i in range(1001): # 723 * 1000 + remain[320] = 723,320 -> # of evaluate samples
            tmp_X = preprocessing_payload(X[i*bs:(i+1)*bs])
            yield tmp_X, y[i*bs:(i+1)*bs]

In [6]:
model = RedRNN()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', Precision(), Recall()])

ear = EarlyStopping(monitor='loss', patience=4)
ckpt = ModelCheckpoint(filepath=r"E:\full_data\RNN_exp.h5", monitor='loss',
                       verbose=1, save_best_only=True)

In [9]:
model.fit_generator(myGenerator(), steps_per_epoch=1001, epochs = 16, verbose=1, callbacks=[ear, ckpt], validation_data=None, class_weight=None, workers=1, shuffle=False)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 1001 steps
Epoch 1/16
1000/1001 [============================>.] - ETA: 1s - loss: 0.6845 - acc: 0.6286 - precision: 0.5675 - recall: 0.5037
Epoch 00001: loss improved from inf to 0.68458, saving model to E:\full_data\RNN_exp.h5
1001/1001 [==============================] - 1170s 1s/step - loss: 0.6847 - acc: 0.6285 - precision: 0.5675 - recall: 0.5035
Epoch 2/16
1000/1001 [============================>.] - ETA: 1s - loss: 0.6786 - acc: 0.5941 - precision: 0.5252 - recall: 0.3974
Epoch 00002: loss improved from 0.68458 to 0.67867, saving model to E:\full_data\RNN_exp.h5
1001/1001 [==============================] - 1177s 1s/step - loss: 0.6788 - acc: 0.5940 - precision: 0.5252 - recall: 0.3973
Epoch 3/16
1000/1001 [============================>.] - ETA: 1s - loss: 0.6697 - acc: 0.6110 - precision: 0.5621 - recall: 0.3538
Epoch 00003: loss improved from 0.67867 to 0.66975, saving

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
now_pay (InputLayer)         [(None, 1600, 257)]       0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                37120     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 37,153
Trainable params: 37,153
Non-trainable params: 0
_________________________________________________________________
